In [ ]:
# External imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st
import itertools
import pickle as pk

# Machine learning
import xgboost as xgb
from sklearn import linear_model, tree
from sklearn.metrics import log_loss, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold

In [ ]:
# Internal imports
import importlib
import extraction as ex
import features as feat
ex = importlib.reload(ex)
feat = importlib.reload(feat)

In [ ]:
# Read data

# Small tables
train = ex.read_train()
test = ex.read_test()
members = ex.read_members()
useful_msno = set.union(
    set(train.index.unique()),
    set(test.index.unique())
)

In [ ]:
# Transactions table
transactions = ex.read_transactions(useful_msno=useful_msno, max_lines=10**7, chunk_size=10**6)
# For train set, pretend we don't know what happens in March
transactions_train = transactions[transactions["transaction_date"] < pd.Timestamp(2017, 3, 1)]

In [ ]:
# EXPLOIT MEMBERS

def exploit_members(members):
    
    # Registration init
    registration_init = feat.count_days(members["registration_init_time"], base_date=pd.Timestamp(2000, 1, 1))

    # Sum up members features
    members_features = pd.DataFrame(index=members.index)
    members_features["registration_init_time"] = registration_init
    
    return members_features

In [ ]:
# EXPLOIT TRANSACTIONS

def exploit_transactions(transactions, dataset="train"):

    # Grouping transactions
    grouped_trans = transactions.groupby("msno")
    # Groupby indices
    trans_msno = grouped_trans.max().index

    # Latest transaction line for each user
    last_trans_indices = (transactions['transaction_date'] == grouped_trans['transaction_date'].transform(max))
    last_transactions = transactions[last_trans_indices].drop_duplicates("msno")
    last_transactions.index = last_transactions["msno"]
    # Reindex with the same indices as the groupby
    last_transactions = last_transactions.reindex(trans_msno).drop("msno", axis=1)
    
    # Latest transaction
    last_trans_date = feat.count_days(grouped_trans["transaction_date"].max(), dataset)
    # Planned expiration
    last_expiration = feat.count_days(grouped_trans["membership_expire_date"].max(), dataset)
    # Number of transactions
    count_trans = grouped_trans["transaction_date"].count()

    # Mean plan days
    mean_plan_days = grouped_trans["payment_plan_days"].mean()
    # Total plan days
    total_plan_days = grouped_trans["payment_plan_days"].sum()
    # Last plan days
    last_plan_days = last_transactions["payment_plan_days"]

    # Freq auto-renew
    freq_auto_renew = grouped_trans["is_auto_renew"].mean()
    # Last auto-renew
    last_auto_renew = last_transactions["is_auto_renew"]

    # Freq cancel
    freq_cancel = grouped_trans["is_cancel"].mean()
    # Last cancel
    last_cancel = last_transactions["is_cancel"]
    # Exist cancel
    exist_cancel = (grouped_trans["is_cancel"].min() > 0).astype(np.int8)

    # Mean price
    mean_price = grouped_trans["actual_amount_paid"].sum()
    # Total price
    total_price = grouped_trans["actual_amount_paid"].sum()

    # Majority payment method
    # majority_payment_method = grouped_trans["payment_method_id"].agg(lambda x:x.value_counts().index[0])
    # Last payment method
    last_payment_method = last_transactions["payment_method_id"]

    # Add all of this to the features table
    transactions_features = pd.DataFrame(index=trans_msno)
    transactions_features["Last_transaction_date"] = last_trans_date
    transactions_features["Planned_membership_expiration"] = last_expiration
    transactions_features["Count_transactions"] = count_trans
    transactions_features["Mean_plan_days"] = mean_plan_days
    transactions_features["Total_plan_days"] = total_plan_days
    transactions_features["Last_plan_days"] = last_plan_days
    transactions_features["Freq_auto_renew"] = freq_auto_renew
    transactions_features["Last_auto_renew"] = last_auto_renew
    transactions_features["Freq_cancel"] = freq_cancel
    transactions_features["Last_cancel"] = last_cancel
    transactions_features["Exist_cancel"] = exist_cancel
    transactions_features["Total_price"] = total_price
    # transactions_features["Majority_payment_method"] = majority_payment_method
    transactions_features["Last_payment_method"] = last_payment_method
    transactions_features = feat.categorize(transactions_features, "Last_payment_method")
    
    return transactions_features

In [ ]:
# EXPLOIT USER LOGS

def exploit_user_logs(user_logs, dataset="train"):
    
    if dataset == "train":
        user_logs_last_month = user_logs[
            (user_logs["date"] > pd.Timestamp(2017, 2, 1)) &
            (user_logs["date"] < pd.Timestamp(2017, 3, 1))
        ]
    elif dataset == "test":
        user_logs_last_month = user_logs[
            (user_logs["date"] > pd.Timestamp(2017, 3, 1)) &
            (user_logs["date"] < pd.Timestamp(2017, 4, 1))
        ]
    
    # Grouping user logs
    grouped_logs = user_logs.groupby("msno")
    grouped_logs_last_month = user_logs_last_month.groupby("msno")
    # Storing indices
    logs_msno = grouped_logs.max().index

    # First log
    first_log = feat.count_days(grouped_logs["date"].min(), dataset)
    # Last log
    last_log = feat.count_days(grouped_logs["date"].max(), dataset)

    # Number of logs
    total_logs = grouped_logs["date"].count()
    total_logs_last_month = grouped_logs_last_month["date"].count()

    logs_features = pd.DataFrame(index = logs_msno)
    logs_features["Last_log"] = last_log
    logs_features["First_log"] = first_log
    logs_features["Total_logs"] = total_logs
    logs_features["Total_logs_last_month"] = total_logs_last_month
    
    if len(user_logs.columns) > 2:
        
        # Total unique songs
        total_unique_songs = grouped_logs["num_unq"].sum()
        total_unique_songs_last_month = grouped_logs_last_month["num_unq"].sum()
        # Total 100% songs
        total_100_songs = grouped_logs["num_100"].sum()
        total_100_songs_last_month = grouped_logs_last_month["num_100"].sum()
        # Total seconds
        total_seconds = grouped_logs["total_secs"].sum()
        total_seconds_last_month = grouped_logs_last_month["total_secs"].sum()

        logs_features["Total_unique_songs"] = total_unique_songs
        logs_features["Total_unique_songs_last_month"] = total_unique_songs_last_month
        logs_features["Total_100_songs"] = total_100_songs
        logs_features["Total_100_songs_last_month"] = total_100_songs_last_month
        logs_features["Total_seconds"] = total_seconds
        logs_features["Total_seconds_last_month"] = total_seconds_last_month
    
    return logs_features

In [ ]:
def add_chunk_to_logs_features(logs_chunk, logs_features, max_lag):
    """Extract features from a chunk of the user_logs and save them."""
    if len(logs_chunk) == 0:
        return
    # Collect features on the chunk
    logs_chunk_features = exploit_user_logs(logs_chunk)
    # Adjust the index on that of all useful users
    # (introducing NaN for the ones not present in the chunk)
    logs_chunk_features = logs_chunk_features.reindex(useful_msno)

    logs_features["Last_log"] = np.minimum(
        logs_features["Last_log"],
        logs_chunk_features["Last_log"].fillna(max_lag))
    logs_features["First_log"] = np.maximum(
        logs_features["First_log"],
        logs_chunk_features["Last_log"].fillna(-max_lag))
    logs_features["Total_logs"] += logs_chunk_features["Total_logs"].fillna(0)
    logs_features["Total_logs_last_month"] += logs_chunk_features["Total_logs_last_month"].fillna(0)

In [ ]:
def read_user_logs_features(
    useful_msno, dataset,
    starting_date=None, just_date=False,
    max_lines=np.inf, chunk_size=10**6
):
    """Read user logs, extract features on the fly."""
    print("\nREADING USER LOGS WITH FEATURE EXTRACTION FOR TRAIN & TEST\n")
    
    global_path = "/tmp/kaggle/proj2/"
    
    logs_features = pd.DataFrame(index=useful_msno)
    logs_train_features = pd.DataFrame(index=useful_msno)
    
    max_lag = 10000
    for df in [logs_features, logs_train_features]:
        # Initialize values
        df["Total_logs"] = 0
        df["Total_logs_last_month"] = 0
        # Days counted backwards from today
        df["Last_log"] = max_lag
        df["First_log"] = -max_lag

    dtype_cols_user_logs = {
        'msno': object,
        'date': np.int64,
        'num_25': np.int8,
        'num_50': np.int8,
        'num_75': np.int8,
        'num_985': np.int8,
        'num_100': np.int8,
        'num_unq': np.int8,
        'total_secs': np.float32
    }

    if just_date:
        usecols=["date", "msno"]
    else:
        usecols=["date", "msno", "num_100", "num_unq", "total_secs"]

    # Read all columns
    iterator1 = pd.read_csv(
        global_path + "data/user_logs.csv",
        chunksize=chunk_size,
        iterator=True,
        header=0,
        dtype=dtype_cols_user_logs,
        usecols=usecols
    )
    iterator2 = pd.read_csv(
        global_path + "data/user_logs_v2.csv",
        chunksize=chunk_size,
        iterator=True,
        header=0,
        dtype=dtype_cols_user_logs,
        usecols=usecols
    )

    # Read data by chunks to alleviate memory load
    chunk_number = 0
    for logs_chunk in itertools.chain(iterator1, iterator2):

        # Convert dates to pandas format
        logs_chunk["date"] = pd.to_datetime(logs_chunk["date"].astype(str))
        # Filter dates for train set
        logs_chunk_train = logs_chunk[logs_chunk["date"] < pd.Timestamp(2017, 3, 1)]
        
        # Add the new information from this chunk to both features tables
        add_chunk_to_logs_features(logs_chunk, logs_features, max_lag)
        add_chunk_to_logs_features(logs_chunk_train, logs_train_features, max_lag)
        
        print("Chunk {} of user logs read".format(chunk_number + 1))
        chunk_number += 1
        if chunk_number >= max_lines / chunk_size:
            break
        
    # Get rid of users for which we haven't read anything
    logs_features = logs_features.replace([max_lag, -max_lag], np.NaN).dropna()
    logs_train_features = logs_train_features.replace([max_lag, -max_lag], np.NaN).dropna()

    return logs_train_features, logs_features



In [ ]:
# Extract features from preexisting tables
members_features = exploit_members(members)
transactions_train_features = exploit_transactions(transactions_train, dataset="train")
transactions_features = exploit_transactions(transactions, dataset="test")

In [ ]:
# Extract features from user_logs while reading it
logs_train_features, logs_features = read_user_logs_features(
    useful_msno=useful_msno, dataset="train",
    starting_date=None, just_date=False,
    max_lines=10**7, chunk_size=10**6
)

In [ ]:
# Save the features to csv to get them back easily
members_features.to_csv("/tmp/kaggle/proj2/junk/members_features.csv")
transactions_train_features.to_csv("/tmp/kaggle/proj2/junk/transactions_train_features.csv")
transactions_features.to_csv("/tmp/kaggle/proj2/junk/transactions_features.csv")
logs_train_features.to_csv("/tmp/kaggle/proj2/junk/logs_train_features.csv")
logs_features.to_csv("/tmp/kaggle/proj2/junk/logs_features.csv")

In [ ]:
members_features = pd.read_csv(
    "/tmp/kaggle/proj2/junk/members_features.csv",
    index_col=0
)
transactions_train_features = pd.read_csv(
    "/tmp/kaggle/proj2/junk/transactions_train_features.csv",
    index_col=0
)
transactions_features = pd.read_csv(
    "/tmp/kaggle/proj2/junk/transactions_features.csv",
    index_col=0
)
logs_train_features = pd.read_csv(
    "/tmp/kaggle/proj2/junk/logs_train_features.csv",
    index_col=0
)
logs_features = pd.read_csv(
    "/tmp/kaggle/proj2/junk/logs_features.csv",
    index_col=0
)

In [ ]:
# Add the data to the train set and test dataframes
features_list_train = [members_features, transactions_train_features, logs_train_features]
features_list = [members_features, transactions_features, logs_train_features]
# The inner join keeps only the users present in every table
train_full = train.join(features_list_train, how="inner")
test_full = test.join(features_list, how="inner")

In [ ]:
# Keep only the features we want
features = test_full.columns # [c for c in test_full.columns if not "payment_method" in c]
train_filtered, test_filtered = feat.select_features(train_full, test_full, features)

# Normalize the columns
train_filtered, test_filtered = feat.normalize_features(train_filtered, test_filtered)

In [ ]:
def log_loss_score_func(y_true, y_pred):
    return log_loss(
        y_true, y_pred, labels=[0, 1],
        eps=np.power(10., -15), normalize=True)

log_loss_scorer = make_scorer(
    score_func=log_loss_score_func,
    greater_is_better=False,
    needs_proba=True
)

In [ ]:
# Here comes the machine learning

# Conversion into arrays for scikit-learn
x = np.array(train_filtered.drop("is_churn", axis=1))
y = np.array(train_filtered["is_churn"])
xt = np.array(test_filtered)

# Initialize xgboost (with arbitrary parameters for now)
xgbclf = xgb.XGBClassifier()

# Perform cross-validation
scores = cross_val_score(
    xgbclf, x, y,
    scoring="neg_log_loss",
    cv=3, n_jobs=1,
    verbose=3,
    fit_params={"eval_metric": "logloss"}
)
print("Cross-validation score", np.mean(scores.mean())

# Train the classifier on all of the data
xgbclf.fit(x, y, eval_metric="logloss")
# Perform the prediction
yt = xgbclf.predict_proba(xt)[:, 1]

In [ ]:
# Parameter selection for xgboost

x = np.array(train_filtered.drop("is_churn", axis=1))
y = np.array(train_filtered["is_churn"])
xt = np.array(test_filtered)

xgbclf = xgb.XGBClassifier()

k = 3
param_grid = {
    "n_estimators": np.round(0.5 * np.power(10, np.linspace(2, 3, k))).astype(int),
    "max_depth": np.arange(3, 10, (10-3) // k),
    "learning_rate": np.power(10, np.linspace(-1.5, -0.2, k))
}

total_tasks=1
for k in param_grid:
    total_tasks *= len(param_grid[k])
print("Total tasks for grid search :", total_tasks, "\n")

gs = GridSearchCV(
    estimator=xgbclf,
    param_grid=param_grid,
    cv=2,
    scoring="neg_log_loss",
    n_jobs=1, verbose=3)

gs.fit(x, y, eval_metric="logloss")  
best_xgbclf = gs.best_estimator_

# pk.dump(gs.cv_results_, open("GS_results", "wb"))
# pk.dump(gs.best_estimator_, open("GS_estimator", "wb"))

In [ ]:
# Use best classifier for prediction
best_xgbclf.fit(x, y, eval_metric="logloss")
yt = best_xgbclf.predict_proba(xt)[:, 1]

# Zero prediction as baseline
percentage_churn = train_filtered["is_churn"].sum() / len(train_filtered)
test["is_churn"] = np.random.rand(len(test)) * percentage_churn
# For users on which we have more info, use it
test.loc[test_filtered.index, ["is_churn"]] = yt.reshape(-1, 1)

# Save as csv
submission = test.loc[:, ["is_churn"]]
submission.to_csv("data/submission.csv")